In [1]:
import sys
sys.path.insert(0, '../src')
pickleFolder = '../PickledData/'
pickleFile = 'preProcessedData.p'
import file_operations



(restaurantTrainDict,restaurantTestDict,laptopTrainDict, laptopTestDict)\
    = file_operations.loadAndGetRawDataFromPickle(pickleFolder, pickleFile)
rst_train_sentences = restaurantTrainDict['sentences']['sentence']
lptp_train_sentences = laptopTrainDict['sentences']['sentence']

all data loaded in python dicts


In [14]:
len(rst_train_sentences[1]['aspectCategories']['aspectCategory'])


2

In [20]:
rst_train_sentences[1]['aspectCategories']

{'aspectCategory': [{'@category': 'food', '@polarity': 'positive'},
  {'@category': 'anecdotes/miscellaneous', '@polarity': 'negative'}]}

In [37]:
one_category_c = 0
one_cate_one_asp = 0
one_cate_multi_asp = 0

multi_cate = 0
multi_cate_one_asp = 0
multi_cate_multi_asp = 0
for sent in rst_train_sentences:
    categories = sent['aspectCategories']['aspectCategory']
    if type(categories) is dict:
        one_category_c += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                one_cate_one_asp += 1
            else:
                one_cate_multi_asp += 1
    elif type(categories) is list:
        # print(categories)
        multi_cate += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                multi_cate_one_asp += 1
            else:
                multi_cate_multi_asp += 1
        
print("one category: {0}".format(one_category_c))
print('one category and one aspect: {0}'.format(one_cate_one_asp))
print('one category and multi aspects: {0}'.format(one_cate_multi_asp))
print('one category and No aspect: {0}'.format(one_category_c - one_cate_one_asp - one_cate_multi_asp))

print('multi categories: {0}'.format(multi_cate))
print('multi categories and one aspect: {0}'.format(multi_cate_one_asp))
print('multi categories and multi aspects: {0}'.format(multi_cate_multi_asp))
print('multi categories and No aspect: {0}'.format(multi_cate - multi_cate_one_asp - multi_cate_multi_asp))






one category: 2468
one category and one aspect: 886
one category and multi aspects: 593
one category and No aspect: 989
multi categories: 576
multi categories and one aspect: 138
multi categories and multi aspects: 406
multi categories and No aspect: 32


In [22]:
counter

2468

In [33]:
rst_train_sentences[4]['aspectTerms']['aspectTerm']

KeyError: 'aspectTerms'

In [38]:
def put_asp_cate(aspect, category, collector):
    """
    
    """
    if aspect in collector:
        cate_dict = collector[aspect]
        if category in cate_dict:
            cate_dict[category] += 1
        else:
            cate_dict[category] = 1
    else:
        collector[aspect] = {}
        collector[aspect][category] = 1

In [41]:
asp_category = {}
for sent in rst_train_sentences:
    categories = sent['aspectCategories']['aspectCategory']
    if type(categories) is dict:
        if 'aspectTerms' in sent:
            
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
#                 print(aspects)
                put_asp_cate(aspects['@term'], categories['@category'], asp_category)
            else:
                for asp in aspects:
                    put_asp_cate(asp['@term'], categories['@category'], asp_category)
    # contains multiple categories
    elif type(categories) is list:
        multi_cate += 1
        if 'aspectTerms' in sent:
            # print(sent['aspectTerms'])
            aspects = sent['aspectTerms']['aspectTerm']
            if type(aspects) is dict:
                for cate in categories:
                    put_asp_cate(aspects['@term'], cate['@category'], asp_category)
            else:
                for cate in categories:
                    for asp in aspects:
                        put_asp_cate(asp['@term'], cate['@category'], asp_category)


In [45]:
asp_category['menu']

{'ambience': 6,
 'anecdotes/miscellaneous': 8,
 'food': 50,
 'price': 5,
 'service': 9}

In [43]:
len(asp_category)

1295

In [50]:
asp2category = {}
for asp in asp_category:
    categories = list(asp_category[asp].keys())
    most_freq_category = categories[0]
    for cate in categories:
        if asp_category[asp][cate] > asp_category[asp][most_freq_category]:
            most_freq_category = cate
    asp2category[asp] = most_freq_category


In [52]:
len(asp2category)

1295